In [1]:
#Import packages
import os
import glob
import json
#from tqdm import tqdm

import nilearn
import nibabel as nib
from nilearn import image as nimg
from nilearn import plotting as nplot
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix, run_glm
from nilearn.glm import fdr_threshold,threshold_stats_img
from nilearn.glm.contrasts import compute_contrast


from bids.layout import BIDSLayout, parse_file_entities

# import cortex
# from cortex import fmriprep

from nipype.interfaces.workbench.base import WBCommand
from nipype.algorithms import modelgen
from nipype.interfaces.base import Bunch

import hcp_utils as hcp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm

from PIL import Image

import first_level_sparse_scanning as flss 
import importlib


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [90]:
root = '../../derivatives/events_sweep'
sub = 'voice980'
ses = '1'
task = 'nwr'
run = '1'
out_dir = 'test'

original_events_file = f'{root}/sub-{sub}_ses-{ses}_task-{task}_run-0{run}_events.tsv'

bids_item = [parse_file_entities(original_events_file)]
fmriprep_dir = '../../derivatives/fmriprep'

importlib.reload(flss)
events = flss.convolve_sparse_scan_glm_with_cifti(bids_item, out_dir)

started sub-voice980_ses-1_task-nwr_rec-unco_run-1
finished sub-voice980_ses-1_task-nwr_rec-unco_run-1


In [92]:
events

['/nese/mit/group/sig/om_projects/voice/bids/data/sub-voice980/ses-1/func/sub-voice980_ses-1_task-nwr_run-02_events.tsv']

In [91]:
!ls test

fails.txt
sub-voice980_ses-1_task-nwr_rec-unco_run-1_space-fsLR_den-91k_contrast-spchsil_effect_size.dscalar.nii
sub-voice980_ses-1_task-nwr_rec-unco_run-1_space-fsLR_den-91k_contrast-spchsil_effect_variance.dscalar.nii
sub-voice980_ses-1_task-nwr_rec-unco_run-1_space-fsLR_den-91k_contrast-spchsil_z_score.dscalar.nii


In [84]:
run_order = flss.return_run_order('../../derivatives/fmriprep/', include_manual=True)

In [86]:
run_order

{'sub-voice844_task-nwr': ['1', '2', '3'],
 'sub-voice844_task-pataka': ['1', '2'],
 'sub-voice844_task-emosent': ['1', '2'],
 'sub-voice844_task-vowel': ['1', '2'],
 'sub-voice846_task-nwr': ['1', '2', '3'],
 'sub-voice846_task-pataka': ['1', '2'],
 'sub-voice846_task-emosent': ['1', '2'],
 'sub-voice846_task-vowel': ['1', '2'],
 'sub-voice849_task-nwr': ['1', '2', '3'],
 'sub-voice849_task-pataka': ['1', '2'],
 'sub-voice849_task-emosent': ['1', '2'],
 'sub-voice849_task-vowel': ['1', '2'],
 'sub-voice850_task-nwr': ['1', '2', '3'],
 'sub-voice850_task-pataka': ['1', '2'],
 'sub-voice850_task-emosent': ['1', '2'],
 'sub-voice850_task-vowel': ['1', '2'],
 'sub-voice852_task-nwr': ['1', '2', '3'],
 'sub-voice852_task-pataka': ['1', '2'],
 'sub-voice852_task-emosent': ['1', '2'],
 'sub-voice852_task-vowel': ['1', '2'],
 'sub-voice854_task-nwr': ['1', '2'],
 'sub-voice854_task-pataka': ['1', '2'],
 'sub-voice854_task-emosent': ['1', '2'],
 'sub-voice854_task-vowel': ['1', '3'],
 'sub-voi

In [12]:
sorted(run_order['sub-voice857_task-vowel']) == run_order['sub-voice857_task-vowel']

True

In [22]:
run_order['sub-voice961_task-nwr']

['3', '2', '1']

In [43]:
s='voice961'
ro = {}
base = f'/nese/mit/group/sig/om_projects/voice/rawData/{s}'
#tasks_included = ['nwr','pataka', 'emosent', 'vowel']
tasks_included = ['nwr']

for sd in next(os.walk(base))[1]:
    if 'behavioral' in os.listdir(base + '/'+ sd):
        for task in tasks_included:
            psychopy_logs = sorted(glob.glob(f'{base}/{sd}/behavioral/*{task}*.psydat'))
            run_numbers = [r.split('_R00')[1].split(f'_{task}')[0] for r in psychopy_logs]
            ro[f'sub-{s}_task-{task}'] = run_numbers


In [44]:
psychopy_logs

['/nese/mit/group/sig/om_projects/voice/rawData/voice961/session001_visit003/behavioral/20161116190800_voice961b_S001_T003_R001_nwr.psydat',
 '/nese/mit/group/sig/om_projects/voice/rawData/voice961/session001_visit003/behavioral/20161116191201_voice961b_S001_T003_R002_nwr.psydat',
 '/nese/mit/group/sig/om_projects/voice/rawData/voice961/session001_visit003/behavioral/20161116191553_voice961b_S001_T003_R003_nwr.psydat']

In [35]:
a = [glob.glob(f'{base}/{sd}/behavioral/*{task}*.psydat') for sd in next(os.walk(base))[1]]
[sorted(b) for b in a if b]

[['/nese/mit/group/sig/om_projects/voice/rawData/voice980/session001_visit001/behavioral/20150320140452_voice_ID980_S001_T003_R002_nwr.psydat',
  '/nese/mit/group/sig/om_projects/voice/rawData/voice980/session001_visit001/behavioral/20150320140845_voice_ID980_S001_T003_R003_nwr.psydat',
  '/nese/mit/group/sig/om_projects/voice/rawData/voice980/session001_visit001/behavioral/20150320145157_voice_ID980_S001_T003_R001_nwr.psydat']]

In [26]:
for b in a:
    if b:
        print(b)

['/nese/mit/group/sig/om_projects/voice/rawData/voice961/session001_visit003/behavioral/20161116190219_voice961b_S001_T006_R002_vowel.psydat', '/nese/mit/group/sig/om_projects/voice/rawData/voice961/session001_visit003/behavioral/20161116185701_voice961b_S001_T006_R001_vowel.psydat']


In [ ]:
glob.glob()

In [19]:
sd

'session001_visit003'

In [16]:
ro

{'sub-voice961_task-nwr': ['3', '2', '1'],
 'sub-voice961_task-pataka': ['1', '2'],
 'sub-voice961_task-emosent': ['1', '2'],
 'sub-voice961_task-vowel': ['2', '1']}

In [22]:
bids_item

[{'subject': 'voice844',
  'session': '1',
  'task': 'nwr',
  'run': 02,
  'suffix': 'events',
  'extension': '.tsv'}]

In [5]:
events

['/nese/mit/group/sig/om_projects/voice/bids/data/sub-voice844/ses-1/func/sub-voice844_ses-1_task-nwr_run-02_events.tsv']

In [ ]:
# TR=task_json['RepetitionTime']
# DT=task_json['DelayTime']

# sparse_model = modelgen.SpecifySparseModel()
# sparse_model.inputs.input_units = 'secs'
# sparse_model.inputs.functional_runs = nifti
# sparse_model.inputs.time_repetition = TR #sets TR to the full window of MRI acq and bold signal [silent+speak]
# sparse_model.inputs.time_acquisition = TR - DT # specifies MR acquisition time
# sparse_model.inputs.scan_onset = TR - DT #specifies onset since acqusition happens first (is this right)
# #sparse_model.inputs.high_pass_filter_cutoff = 128. #satra said to remove
# sparse_model.inputs.model_hrf = True
# sparse_model.inputs.subject_info = modelgen.bids_gen_info(events,condition_column='trial_type')  # doctest: +SKIP

# #potentially could also tack on the noise regressors too here
# regressors = sparse_model._list_outputs()['session_info'][0]['regress']
# data = [v['val'] for v in regressors]
# col = [t['name'] for t in regressors]
# df_regressors = pd.DataFrame(data).T
# df_regressors.columns = col
# df_regressors.index = frame_times